In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

# Upload the CSV file
print("📁 Please upload your solar_panel_dust_data.csv file")
uploaded = files.upload()

📁 Please upload your solar_panel_dust_data.csv file


TypeError: 'NoneType' object is not subscriptable

In [ ]:


# Load the data
filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)

print(f"\n Data loaded successfully!")
print(f"Original shape: {df.shape}")
print(f"\nOriginal columns: {list(df.columns)}")

# Display first few rows
print("\n First 5 rows of original data:")
print(df.head())

# =======================
# DATA CLEANING PROCESS
# =======================

print("\n" + "="*60)
print("STARTING DATA CLEANING PROCESS")
print("="*60)

# Step 1: Remove panel_type column
print("\n  Step 1: Removing 'panel_type' column...")
if 'panel_type' in df.columns:
    df = df.drop('panel_type', axis=1)
    print(" 'panel_type' column removed")
else:
    print("  'panel_type' column not found")

print(f"New columns: {list(df.columns)}")

# Step 2: Check for missing values
print("\n Step 2: Checking for missing values...")
missing_values = df.isnull().sum()
missing_percent = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percent
})
print(missing_df[missing_df['Missing Values'] > 0])

if missing_values.sum() > 0:
    print(f"\n  Total missing values: {missing_values.sum()}")
else:
    print("\n No missing values found")

# Step 3: Handle missing values
print("\n Step 3: Handling missing values...")
initial_rows = len(df)

# Remove rows where critical columns have missing values
critical_columns = ['panel_id', 'latitude', 'longitude', 'dust_accumulation', 'efficiency_drop']
df = df.dropna(subset=critical_columns)

# Fill missing values in non-critical columns with median
numeric_columns = ['sunlight_hours', 'temperature', 'humidity', 'wind_speed', 'rainfall']
for col in numeric_columns:
    if col in df.columns and df[col].isnull().sum() > 0:
        median_value = df[col].median()
        df[col].fillna(median_value, inplace=True)
        print(f" Filled {col} missing values with median: {median_value:.2f}")

rows_removed = initial_rows - len(df)
if rows_removed > 0:
    print(f"  Removed {rows_removed} rows with missing critical values")

# Step 4: Check for duplicates
print("\n Step 4: Checking for duplicates...")
duplicates = df.duplicated(subset=['panel_id']).sum()
if duplicates > 0:
    print(f"  Found {duplicates} duplicate panel_ids")
    df = df.drop_duplicates(subset=['panel_id'], keep='first')
    print(f" Removed {duplicates} duplicate records")
else:
    print(" No duplicates found")

# Step 5: Data type validation
print("\n Step 5: Validating data types...")
print(df.dtypes)

# Convert numeric columns if needed
for col in numeric_columns + ['dust_accumulation', 'efficiency_drop', 'latitude', 'longitude']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Step 6: Statistical summary
print("\n Step 6: Statistical Summary")
print(df.describe())

# Step 7: Detect outliers using IQR method
print("\n Step 7: Detecting outliers (IQR method)...")
numeric_cols = df.select_dtypes(include=[np.number]).columns

outlier_info = {}
for col in numeric_cols:
    if col not in ['panel_id']:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        outlier_count = len(outliers)

        if outlier_count > 0:
            outlier_info[col] = {
                'count': outlier_count,
                'percentage': (outlier_count / len(df)) * 100,
                'lower_bound': lower_bound,
                'upper_bound': upper_bound
            }
            print(f"⚠️  {col}: {outlier_count} outliers ({outlier_info[col]['percentage']:.2f}%)")
            print(f"   Valid range: [{lower_bound:.2f}, {upper_bound:.2f}]")

if not outlier_info:
    print("No significant outliers detected")

# Step 8: Data quality checks
print("\n🔍 Step 8: Data quality checks...")

# Check latitude range (-90 to 90)
invalid_lat = df[(df['latitude'] < -90) | (df['latitude'] > 90)]
if len(invalid_lat) > 0:
    print(f"  Found {len(invalid_lat)} rows with invalid latitude")
    df = df[(df['latitude'] >= -90) & (df['latitude'] <= 90)]

# Check longitude range (-180 to 180)
invalid_lon = df[(df['longitude'] < -180) | (df['longitude'] > 180)]
if len(invalid_lon) > 0:
    print(f"  Found {len(invalid_lon)} rows with invalid longitude")
    df = df[(df['longitude'] >= -180) & (df['longitude'] <= 180)]

# Check negative values where they shouldn't exist
for col in ['sunlight_hours', 'temperature', 'humidity', 'wind_speed', 'rainfall', 'dust_accumulation']:
    if col in df.columns:
        negative_count = (df[col] < 0).sum()
        if negative_count > 0:
            print(f"  Found {negative_count} negative values in {col}")

# Final summary
print("\n" + "="*60)
print("CLEANING COMPLETE!")
print("="*60)
print(f" Original rows: {initial_rows}")
print(f" Final rows: {len(df)}")
print(f" Rows removed: {initial_rows - len(df)}")
print(f" Final columns: {len(df.columns)}")
print(f" Data quality: {(len(df)/initial_rows)*100:.2f}% retained")

# Display cleaned data info
print("\n Cleaned Data Information:")
print(df.info())

print("\n First 5 rows of cleaned data:")
print(df.head())

# Visualizations
print("\n Generating visualizations...")

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Distribution of dust accumulation
axes[0, 0].hist(df['dust_accumulation'], bins=30, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Distribution of Dust Accumulation')
axes[0, 0].set_xlabel('Dust Accumulation')
axes[0, 0].set_ylabel('Frequency')

# 2. Distribution of efficiency drop
axes[0, 1].hist(df['efficiency_drop'], bins=30, color='coral', edgecolor='black')
axes[0, 1].set_title('Distribution of Efficiency Drop')
axes[0, 1].set_xlabel('Efficiency Drop')
axes[0, 1].set_ylabel('Frequency')

# 3. Scatter plot: Dust vs Efficiency
axes[1, 0].scatter(df['dust_accumulation'], df['efficiency_drop'], alpha=0.5, color='green')
axes[1, 0].set_title('Dust Accumulation vs Efficiency Drop')
axes[1, 0].set_xlabel('Dust Accumulation')
axes[1, 0].set_ylabel('Efficiency Drop')

# 4. Missing values heatmap (if any remain)
missing_data = df.isnull().sum()
axes[1, 1].barh(range(len(missing_data)), missing_data.values)
axes[1, 1].set_yticks(range(len(missing_data)))
axes[1, 1].set_yticklabels(missing_data.index)
axes[1, 1].set_title('Missing Values by Column')
axes[1, 1].set_xlabel('Count')

plt.tight_layout()
plt.show()

# Correlation heatmap
plt.figure(figsize=(12, 8))
correlation = df.select_dtypes(include=[np.number]).corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Correlation Heatmap of Numeric Variables')
plt.tight_layout()
plt.show()

# Save cleaned data
print("\n Saving cleaned data...")
cleaned_filename = 'solar_panel_dust_data_cleaned.csv'
df.to_csv(cleaned_filename, index=False)
print(f" Cleaned data saved as: {cleaned_filename}")

# Download the cleaned file
print("\n Downloading cleaned data...")
files.download(cleaned_filename)

print("\n All done! Your data is cleaned and ready for analysis!")